# Global variables

Consider once more the code for the recursive implementation of the Levenshtein distance.

In [ ]:
def construct_grid(s, t):
    """Compute the default cost for each edge."""
    # grid is now a dictionary instead of a list
    grid = {(x,y): {}
            for x in range(len(s) + 1)
            for y in range(len(t) + 1)}
    for x, y in grid:
        # add deletion edge if there is a node to the left
        if x > 0:
            grid[(x,y)][(x-1, y)] = 1
        # add insertion edge is there is a node above
        if y > 0:
            grid[(x,y)][(x, y-1)] = 1
        # add substitution edge; check if cost is 0
        if x > 0 and y > 0:
            grid[(x,y)][(x-1, y-1)] =\
                0 if s[x-1] == t[y-1] else 1
    return grid

In [ ]:
def cost(node, grid):
    """Calculate the cost of the optimal path to node through the grid."""
    if node == (0, 0) or node in memo:
        return memo[node]
    else:
        lowest = min([cost(neighbor, grid) + edge_cost
                      for neighbor, edge_cost in grid[node].items()])
        memo[node] = lowest
        return lowest

In [ ]:
def levenshtein_distance(s, t):
    """Calculate Levenshtein distance between s and t."""
    return cost((len(s), len(t)), construct_grid(s, t))

Notice how `memo` isn't actually instantiated anywhere?
So if we don't take care to instantiate it before calling the function, we'll just get an error.

In [ ]:
pairs = [("fire", "fry"),
         ("fyre", "fry"),
         ("apple", "banana"),
         ("aaa", "bbb"),
         ("long string", "")]
for s, t in pairs:
    print(f"Levenshtein distance of \"{s}\" and \"{t}\" is {levenshtein_distance(s, t)}")

It would be much easier if we could instantiate `memo` as part of the `levenshtein_distance` function.

In [ ]:
def levenshtein_distance(s, t):
    """Calculate Levenshtein distance between s and t."""
    memo = {(0, 0): 0}
    return cost((len(s), len(t)), construct_grid(s, t))

But this doesn't work either.

In [ ]:
pairs = [("fire", "fry"),
         ("fyre", "fry"),
         ("apple", "banana"),
         ("aaa", "bbb"),
         ("long string", "")]
for s, t in pairs:
    print(f"Levenshtein distance of \"{s}\" and \"{t}\" is {levenshtein_distance(s, t)}")

The problem is that variables within a function `f` are only accessible within this function.
They cannot be accessed from outside.
This even holds for functions that are defined outside `f` but are called by `f`.

But why, then, does it work if `memo` is not defined inside a function?
Variables defined outside functions are **global variables**.
A global variable can be accessed by any part of the code any time.
We can tell Python explicitly that a variable is global by using the `global` command.

In [ ]:
def levenshtein_distance(s, t):
    """Calculate Levenshtein distance between s and t."""
    global memo  # memo is a global variable
    memo = {(0, 0): 0}
    return cost((len(s), len(t)), construct_grid(s, t))

In [ ]:
pairs = [("fire", "fry"),
         ("fyre", "fry"),
         ("apple", "banana"),
         ("aaa", "bbb"),
         ("long string", "")]
for s, t in pairs:
    print(f"Levenshtein distance of \"{s}\" and \"{t}\" is {levenshtein_distance(s, t)}")

Now things work the way they should.
But keep in mind that `memo` is still accessible to any arbitrary piece of code.

In [ ]:
print(memo)

This can cause all kinds of problems if you aren't careful.
For instance, another function might expect `memo` to be empty, so now that it's been filled up with values from `levenshtein_distance` this function might misbehave.

To be on the safe side, delete a global variable once you don't need it anymore.
This can be done with `del`.

In [ ]:
def levenshtein_distance(s, t):
    """Calculate Levenshtein distance between s and t."""
    global memo  # memo is a global variable
    memo = {(0, 0): 0}
    distance = cost((len(s), len(t)), construct_grid(s, t))
    del memo  # delete global variable
    return distance

In [ ]:
pairs = [("fire", "fry"),
         ("fyre", "fry"),
         ("apple", "banana"),
         ("aaa", "bbb"),
         ("long string", "")]
for s, t in pairs:
    print(f"Levenshtein distance of \"{s}\" and \"{t}\" is {levenshtein_distance(s, t)}")

In [ ]:
# memo is now undefined
print(memo)

## Bullet point summary

- Variables defined inside a function `f` cannot be accessed from anywhere outside `f`.
- They are also inaccessible to any other functions that are defined outside `f`, even if `f` calls those functions.
- Global variables are accessible from anywhere.
- Use `global var` to define a global variable `var`.
- Make sure to delete global variables with `del var`.
  Otherwise, they might cause weird bugs that are hard to track down.